#Privthi configuration file guide
One essential step of working with the Prithvi HLS foundation model is to set up a configuration file. This file will specify everything from the dataset file paths to training and model metaparameters. The intention of this notebook is to familiarize users with many of the important parameters that require user definition. The configuration defined in this notebook corresponds with [the HLS foundation fine-tuning guide found here](https://colab.research.google.com/drive/1pPhGt9x3p6KVXG4oIUc9y-sqPcaevFbq?usp=sharing).

Before running, ensure that you have completed the well pad data generation by following [this guide](https://colab.research.google.com/drive/1VEJOpyeTJeYSX8EcINe1m7lA-Bi7YoqU?usp=sharing).

##Overview

Each cell within this notebook corresponds to a section of the config file. Each time you run a cell, the code within that section is appended to the file named hls_config.py which is stored in colab's local storage. It is recommended not to run a cell more than once. Each cell will have a title and some will have a brief description.

###Imports and Setup

In [ ]:
%%writefile -a hls_config.py
import os
custom_imports = dict(imports=['geospatial_fm'])

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
cudnn_benchmark = True

dataset_type = 'GeospatialDataset'

Writing hls_config.py


###Data Root
The `data_root` parameter specifies the path to the dataset folder. This folder should contain a training and validation folder. Each of these folders should contain both the mask and image tiles used for training/validation.

In [ ]:
%%writefile -a hls_config.py
data_root = '/content/gdrive/MyDrive/HLS foundation guide/dataset'

Appending to hls_config.py


###GPU settings and Image Specs

In [ ]:
%%writefile -a hls_config.py
num_frames = 1
img_size = 224
num_workers = 4
samples_per_gpu = 4

Appending to hls_config.py


###Mean and Standard Deviation
This is the mean and standard deviation of the pixel values for each band within the whole training set. This information is useful for normalization.

In [ ]:
%%writefile -a hls_config.py
img_norm_cfg = dict(
    means=[0.08285756, 0.09015894, 0.14294316, 0.26300612, 0.25897524,
       0.21503   ],
    stds=[0.03269446, 0.03606708, 0.04953382, 0.07931096, 0.06092547,
       0.05726682])

Appending to hls_config.py


###Image Settings

* `bands` specifies which bands from the training data will be passed into the model during training. The bands that the HLS Privthi model is pre-trained on are [Blue, Green, Red, Narrow NIR, SWIR1, and SWIR2].

* `img_suffix` and `seg_map_suffix` are used to distinguish image tiles from mask tiles. These are the naming conventions used when generating the data.

* `orig_nsize` is the pixel width and height of your tiles.

* `tile_size` are what the tiles are resized to before being passed into the model. It is suggested not to change this value.

In [ ]:
%%writefile -a hls_config.py
bands = [0, 1, 2, 3, 4, 5]
img_suffix = '_tile.tif'
seg_map_suffix = '_mask.tif'
orig_nsize = 256
tile_size = 224

crop_size = (tile_size, tile_size)
ignore_index = -1
image_nodata = -9999
image_nodata_replace = 0
image_to_float32 = True

Appending to hls_config.py


###Experiment name and file path.

The `experiment` variable is the name of the project and `project_dir` is the location where all of the files associated with fine-tuning will be saved. Model checkpoints will be saved here.

In [ ]:
%%writefile -a hls_config.py
experiment = 'HLS_wellpad_finetuning'
project_dir = '/content/gdrive/MyDrive/HLS foundation guide'

work_dir = os.path.join(project_dir, experiment)
save_path = work_dir

Appending to hls_config.py


###Training Parameters and Pre-trained Model path
The `pretrained_weights_path` specifies the location of the pre-trained HLS Prithvi model. Before fine tuning, you must download the Prithvi_100m.pt checkpoint from [this repo](https://huggingface.co/ibm-nasa-geospatial/Prithvi-100M).

The `epochs` and `eval_epoc_interval` specifies how many epochs to train for and when to evaluate model performance on the validation set. For the purpose of this guide, we will train for only 5 epochs and evaluate after each epoch.

In [ ]:
%%writefile -a hls_config.py
pretrained_weights_path =  '/content/gdrive/MyDrive/HLS Foundation Model/Prithvi_100M.pt'
epochs=5
eval_epoch_interval = 1


num_layers = 12
patch_size = 16
embed_dim = 768
num_heads = 12
tubelet_size = 1
output_embed_dim = num_frames*embed_dim

Appending to hls_config.py


###Training, Validation, and Testing pipelines
These lists specify the on-the-fly preprocessing steps that are applied to the data during training, testing, and validation

In [ ]:
%%writefile -a hls_config.py
train_pipeline = [
    dict(type='LoadGeospatialImageFromFile', to_float32=image_to_float32),
    dict(type='LoadGeospatialAnnotations', reduce_zero_label=False),
    dict(type='BandsExtract', bands=bands),
    dict(type='RandomFlip', prob=0.5),
    dict(type='ToTensor', keys=['img', 'gt_semantic_seg']),
    # to channels first
    dict(type="TorchPermute", keys=["img"], order=(2, 0, 1)),
    dict(type='TorchNormalize', **img_norm_cfg),
    dict(type='TorchRandomCrop', crop_size=(tile_size, tile_size)),
    dict(type='Reshape', keys=['img'], new_shape=(len(bands), num_frames, tile_size, tile_size)),
    dict(type='Reshape', keys=['gt_semantic_seg'], new_shape=(1, tile_size, tile_size)),
    dict(
        type='CastTensor',
        keys=['gt_semantic_seg'],
        new_type='torch.LongTensor'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg'])
]
test_pipeline = [
    dict(type='LoadGeospatialImageFromFile', to_float32=image_to_float32),
    dict(type='BandsExtract', bands=bands),
    dict(type='ToTensor', keys=['img']),
    # to channels first
    dict(type="TorchPermute", keys=["img"], order=(2, 0, 1)),
    dict(type='TorchNormalize', **img_norm_cfg),
    dict(
        type='Reshape',
        keys=['img'],
        new_shape=(len(bands), num_frames, -1, -1),
        look_up=dict({
            '2': 1,
            '3': 2
        })),
    dict(type='CastTensor', keys=['img'], new_type='torch.FloatTensor'),
    dict(
        type='CollectTestList',
        keys=['img'],
        meta_keys=[
            'img_info', 'seg_fields', 'img_prefix', 'seg_prefix', 'filename',
            'ori_filename', 'img', 'img_shape', 'ori_shape', 'pad_shape',
            'scale_factor', 'img_norm_cfg'
        ])
]

Appending to hls_config.py


###Training, Testing, and Validation split information
These dictionaries specify several pieces of information that relate to the various dataset splits including folder locations, number of classes, and desired pipeline. For the purposes of this guideline, the `validation` and `test` sets source their data from the same folder.

In [ ]:
%%writefile -a hls_config.py
CLASSES = ('Not Well Pad', 'Well Pad')

data = dict(
    samples_per_gpu=samples_per_gpu,
    workers_per_gpu=num_workers,
    train=dict(
        type=dataset_type,
        CLASSES=CLASSES,
        data_root=data_root,
        img_dir='training',
        ann_dir='training',
        img_suffix=img_suffix,
        seg_map_suffix=seg_map_suffix,
        pipeline=train_pipeline,
        ignore_index=-1),
    val=dict(
        type=dataset_type,
        CLASSES=CLASSES,
        data_root=data_root,
        img_dir='validation',
        ann_dir='validation',
        img_suffix=img_suffix,
        seg_map_suffix=seg_map_suffix,
        pipeline=test_pipeline,
        ignore_index=-1),
    test=dict(
        type=dataset_type,
        CLASSES=CLASSES,
        data_root=data_root,
        img_dir='validation',
        ann_dir='validation',
        img_suffix=img_suffix,
        seg_map_suffix=seg_map_suffix,
        pipeline=test_pipeline,
        ignore_index=-1))


Appending to hls_config.py


###Training Meta-Parameters
This sections specifies other important metaparameters that are relavent for training including the loss function, optimizer, and learning rate.

In [ ]:
%%writefile -a hls_config.py
optimizer = dict(type='Adam', lr=1.3e-05, betas=(0.9, 0.999))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='poly',
    warmup='linear',
    warmup_iters=1500,
    warmup_ratio=1e-06,
    power=1.0,
    min_lr=0.0,
    by_epoch=False)
log_config = dict(
    interval=20,
    hooks=[
        dict(type='TextLoggerHook', by_epoch=False),
        dict(type='TensorboardLoggerHook', by_epoch=False)
    ])
checkpoint_config = dict(
    by_epoch=True, interval=10, out_dir=save_path
)

evaluation = dict(
    interval=eval_epoch_interval, metric="mIoU", pre_eval=True, save_best="mIoU", by_epoch=True
)

runner = dict(type="EpochBasedRunner", max_epochs=epochs)

loss_func=dict(
            type='DiceLoss', use_sigmoid=False, loss_weight=1,
            ignore_index=-1)

Appending to hls_config.py


###Fine-Tune Model Architecture

In order to fine-tune the HLS Prithvi model, several layers are added to the model to allow for image segmentation to occur. Here is where those segmentation layers are defined

In [ ]:
%%writefile -a hls_config.py
workflow = [('train', 1)]
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='TemporalEncoderDecoder',
    frozen_backbone=False,
    backbone=dict(
        type='TemporalViTEncoder',
        pretrained=pretrained_weights_path,
        img_size=img_size,
        patch_size=patch_size,
        num_frames=num_frames,
        tubelet_size=tubelet_size,
        in_chans=len(bands),
        embed_dim=embed_dim,
        depth=12,
        num_heads=num_heads,
        mlp_ratio=4.0,
        norm_pix_loss=False),
    neck=dict(
        type='ConvTransformerTokensToEmbeddingNeck',
        embed_dim=embed_dim*num_frames,
        output_embed_dim=output_embed_dim,
        drop_cls_token=True,
        Hp=14,
        Wp=14),
    decode_head=dict(
        num_classes=len(CLASSES),
        in_channels=output_embed_dim,
        type='FCNHead',
        in_index=-1,
        channels=256,
        num_convs=1,
        concat_input=False,
        dropout_ratio=0.1,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=loss_func),
    auxiliary_head=dict(
        num_classes=len(CLASSES),
        in_channels=output_embed_dim,
        type='FCNHead',
        in_index=-1,
        channels=256,
        num_convs=2,
        concat_input=False,
        dropout_ratio=0.1,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=loss_func),
    train_cfg=dict(),
    test_cfg=dict(mode='slide', stride=(int(tile_size/2), int(tile_size/2)), crop_size=(tile_size, tile_size)))
gpu_ids = range(0, 1)
auto_resume = False

Appending to hls_config.py


###Connect to Google Drive and save the config file in the desired location.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
import shutil

source_folder = '/content/hls_config.py'
destination_folder = '/content/gdrive/MyDrive/HLS foundation guide'

shutil.move(source_folder, destination_folder)

'/content/gdrive/MyDrive/HLS foundation guide/hls_config.py'